# FULL MNIST

In [10]:
#hide
!pip install dataset

In [11]:
#from fastai.vision.all import DataLoader, DataLoaders, vision_learner, resnet18, PILImage, Resize
from fastai.vision.all import *
from fastai.data.core import L
from fastai.metrics import accuracy
import torch.nn.functional as F

In [12]:
from datasets import load_dataset

dataset = load_dataset("mnist")
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [13]:
img3 = dataset['train']['image'][0]
img3.show(), type(img3)

(None, PIL.PngImagePlugin.PngImageFile)

In [50]:
type(PILImage.create(img3))

fastai.vision.core.PILImage

In [14]:
dataset['train']['label'][0], type(dataset['train']['label'][0])

(5, int)

In [15]:
train_ds = dataset['train']
test_ds = dataset['test']
train_ds, test_ds

(Dataset({
     features: ['image', 'label'],
     num_rows: 60000
 }),
 Dataset({
     features: ['image', 'label'],
     num_rows: 10000
 }))

In [16]:
train_ds[0]['label']

5

In [61]:
mnist = DataBlock(
    blocks=(ImageBlock(cls=PILImage), CategoryBlock),
    get_x=lambda x: PILImage.create(x['image']).convert('RGB'),
    get_y=lambda x: x['label'],
    splitter=RandomSplitter(valid_pct=0.2),
)   

In [63]:
dls = mnist.dataloaders(train_ds)
dls

In [64]:
dls.train_ds

(#48000) [(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(6)),(PILImage mode=RGB size=28x28, TensorCategory(4)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(6)),(PILImage mode=RGB size=28x28, TensorCategory(8)),(PILImage mode=RGB size=28x28, TensorCategory(6)),(PILImage mode=RGB size=28x28, TensorCategory(6))...]

In [65]:
learn = vision_learner(dls, resnet18, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)

In [68]:
learn.fit_one_cycle(10, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.263067,31.031359,0.932583,00:34
1,0.996403,781.211853,0.843417,00:34
2,1.248157,48.445045,0.933000,00:33
3,1.109637,1.883233,0.940083,00:32
4,0.737672,238.556839,0.964500,00:33
5,0.464649,0.576921,0.955583,00:33
6,0.217559,0.156954,0.981750,00:33
7,0.081762,0.076060,0.986333,00:33
8,0.022330,0.042081,0.991583,00:33
9,0.015258,0.037065,0.992583,00:33


In [28]:
learn.dls.vocab

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [90]:
learn.validate()

(#2) [0.037064943462610245,0.9925833344459534]

In [94]:
#test_dls = mnist.dataloaders(test_ds)
test_dls = learn.dls.test_dl(test_ds)
preds, _ = learn.get_preds(dl=test_dls)

In [102]:
x = learn.validate(dl=test_dls)

In [107]:
test_ds.__len__()

10000